In [ ]:
import numpy as np
import os
import torch
from torchvision.transforms import functional as F
from torchvision.models.detection import (
    fasterrcnn_resnet50_fpn_v2,
    FasterRCNN_ResNet50_FPN_V2_Weights,
)
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt

class PeopleDetectionAnalyzer:
    def __init__(self):
        self.model = fasterrcnn_resnet50_fpn_v2(weights=FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT, box_score_thresh=0.9)
        self.detection_stats = {
            'frame_count': 0,
            'total_detections': 0,
            'confidences': [],
            'detection_sizes': []
        }
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
    def draw_detections(self, frame, person_boxes, person_scores):
        """Отрисовка bounding box'ов"""

        for box, score in zip(
            person_boxes.cpu().numpy(), person_scores.cpu().numpy()
        ):

            if score > 0.8:
                x1, y1, x2, y2 = map(int, box)
                self.detection_stats['confidences'].append(score)
                self.detection_stats['detection_sizes'].append((x2-x1)*(y2-y1))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
                cv2.putText(
                    frame,
                    f"person: {score:.2f}",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 0),
                    1,
                )
        self.detection_stats['frame_count'] += 1
        self.detection_stats['total_detections'] += len(person_boxes)
        return frame

    def analyze_results(self, output_dir="analysis"):
        """Анализ и визуализация статистики"""
        os.makedirs(output_dir, exist_ok=True)

        # Основные метрики
        avg_conf = np.mean(self.detection_stats['confidences']) if self.detection_stats['confidences'] else 0
        avg_detections_per_frame = self.detection_stats['total_detections'] / max(1, self.detection_stats['frame_count'])

        # Сохранение отчета
        with open(f"{output_dir}/report.txt", "w", encoding='utf-8') as f:
            f.write(f"Всего кадров: {self.detection_stats['frame_count']}\n")
            f.write(f"Всего обнаружений: {self.detection_stats['total_detections']}\n")
            f.write(f"Среднее число людей на кадр: {avg_detections_per_frame:.2f}\n")
            f.write(f"Средняя уверенность: {avg_conf:.2f}\n")
            f.write(f"Минимальная уверенность: {min(self.detection_stats['confidences'], default=0):.2f}\n")
            f.write(f"Максимальная уверенность: {max(self.detection_stats['confidences'], default=0):.2f}\n")

        # Визуализация
        plt.figure(figsize=(15, 5))

        # 1. Распределение уверенности
        plt.subplot(1, 2, 1)
        plt.hist(self.detection_stats['confidences'], bins=20, color='skyblue')
        plt.title("Распределение уверенности детекций")
        plt.xlabel("Уверенность")
        plt.ylabel("Количество")

        # 2. Распределение размеров bbox
        plt.subplot(1, 2, 2)
        plt.hist(self.detection_stats['detection_sizes'], bins=20, color='salmon')
        plt.title("Распределение размеров bbox")
        plt.xlabel("Площадь (пиксели)")
        plt.ylabel("Количество")

        plt.tight_layout()
        plt.savefig(f"{output_dir}/distributions.png")
        plt.close()


In [10]:

def process_video(input_path, output_path):
    detector = PeopleDetectionAnalyzer()
    cap = cv2.VideoCapture(input_path)

    # Параметры видео
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Инициализация VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Обработка видео
    for _ in tqdm(range(total_frames), desc="Обработка видео"):
        ret, frame = cap.read()
        if not ret:
            break
           # convert frame to tensor
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_tensor = F.to_tensor(frame_rgb).to(detector.device)
        detector.model.to(detector.device)
        # run inference
        with torch.no_grad():
            predictions = detector.model([image_tensor])[0]
        predictions
        person_boxes = predictions["boxes"][(predictions["labels"] == 1)]
        person_scores = predictions["scores"][(predictions["labels"] == 1)]
        frame_with_boxes = detector.draw_detections(frame, person_boxes, person_scores)
        out.write(frame_with_boxes)

    # Завершение
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Анализ результатов
    detector.analyze_results()
    print(f"\nАнализ сохранен в папку 'analysis'")


In [17]:
if __name__ == "__main__":
    input_video = "crowd.mp4"
    output_video = "output_videos/detected_crowd.mp4"

    os.makedirs(os.path.dirname(output_video), exist_ok=True)
    process_video(input_video, output_video)

Обработка видео: 100%|██████████| 705/705 [03:19<00:00,  3.53it/s]



Анализ сохранен в папку 'analysis'
